In [1]:
import cv2
import os
import argparse
import glob
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
from models import DnCNN
from utils import *

In [4]:
class options():
    pass

opt = options()
opt.num_of_layers = 17 #"Number of total layers")
opt.logdir = "logs" #'path of log files')
opt.test_data ='Set12' #'test on Set12 or Set68')
opt.test_noiseL =25 #'noise level used on test set')

In [6]:
def normalize(data):
    return data/255.

# Build model
print('Loading model ...\n')
model = DnCNN(channels=1, num_of_layers=opt.num_of_layers)
model.load_state_dict(torch.load(os.path.join(opt.logdir, 'net.pth')))
model.eval()

Loading model ...



IOError: [Errno 2] No such file or directory: 'logs/net.pth'

In [ ]:
print('Loading data info ...\n')
files_source = glob.glob(os.path.join('data', opt.test_data, '*.png'))
files_source.sort()
# process data
psnr_test = 0
for f in files_source:
    # image
    Img = cv2.imread(f)
    Img = normalize(np.float32(Img[:,:,0]))
    Img = np.expand_dims(Img, 0)
    Img = np.expand_dims(Img, 1)
    ISource = torch.Tensor(Img)
    # noise
    noise = torch.FloatTensor(ISource.size()).normal_(mean=0, std=opt.test_noiseL/255.)
    # noisy image
    INoisy = ISource + noise
    ISource, INoisy = Variable(ISource), Variable(INoisy)
    with torch.no_grad(): # this can save much memory
        Out = torch.clamp(INoisy-model(INoisy), 0., 1.)
    
    psnr = batch_PSNR(Out, ISource, 1.)
    psnr_test += psnr
    print("%s PSNR %f" % (f, psnr))
psnr_test /= len(files_source)
print("\nPSNR on test data %f" % psnr_test)